<h1> Libraries </h1>

In [ ]:
import os, sys
import numpy as np
from PIL import Image
import scipy
import moviepy.editor as mpe
from moviepy.video.io.bindings import mplfig_to_npimage

from scipy.sparse.linalg import svds
from matplotlib import pyplot as plt

%matplotlib inline

np.set_printoptions(precision=4, linewidth=100)

<h1> Functions (Create Matrix, Make Videos ...etc) </h1>

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[..., :3], [0.299, 0.587, 0.114])

In [ ]:
def create_data_matrix_from_images(images, scale=50):
    return np.vstack([scipy.misc.imresize(rgb2gray(i).astype(int), scale).flatten() 
                      for i in images]).T

turn background into black(0) and car into white(1)

In [ ]:
def getCar(x):
    xShape = x.shape
    newX = x.copy()
    c = max(0.5,(np.mean(x[:20, :20])+ np.mean(x[:20, xShape[1]-20:]) + np.mean(x[xShape[0]-20:, :20])+ np.mean(x[xShape[0]-20:,xShape[1]-20:]))/4)
    for i in range(xShape[0]):
        for j in range(xShape[1]):
            if abs(x[i][j]-c) < abs(40*c):
                newX[i][j] = 0
            else:
                newX[i][j] = 1
    return newX

def getCar2(x):
    xShape = x.shape
    c = max(0.5,(np.mean(x[:20, :20])+ np.mean(x[:20, xShape[1]-20:]) + np.mean(x[xShape[0]-20:, :20])+ np.mean(x[xShape[0]-20:,xShape[1]-20:]))/4)
    return np.where(abs(x-c)<abs(40*c), 0.0, 1.0)

In [ ]:
def makeVideoFromImgs(videoName, frames, firstAsTime):
    fig, ax = plt.subplots()
    fps = 50
    def make_frame(t):
        ax.clear()
        ax.axis('off')
        ax.set_adjustable('box-forced')
        if firstAsTime:
            ax.imshow(frames[int(t*fps), ...], cmap="gray")
        else:
            ax.imshow(frames[..., int(t*fps)], cmap="gray")
        return mplfig_to_npimage(fig)
    if firstAsTime:
        animation = mpe.VideoClip(make_frame, duration=int(frames.shape[0]/fps))
    else:
        animation = mpe.VideoClip(make_frame, duration=int(frames.shape[-1]/fps))
    animation.write_videofile('./'+videoName+'.mp4', fps=fps)

dice coefficient of truth and target images

In [ ]:
def diceCoef(truth, inp):
    total = 0;
    intersect = 0
    for i in range(len(truth)):
        inpWhite = np.sum(inp[i, ...])
        truthWhite = len(np.where(truth[i]==255.0)[0])
        inters = len(np.where(truth[i]+inp[i,...]==256)[0])
        total += inpWhite + truthWhite
        intersect += inters
        #print(inpWhite, truthWhite, inters)
    return 2*intersect/total

# Video 1

http://bmc.iut-auvergne.com/?page_id=24

111, PNG files with ground truth

# Read Images

In [ ]:
imgs = []
for imgName in range(1,1500):
    temp = np.array(Image.open("./dataset/video1/input/"+str(imgName)+".png"))
    imgs.append(temp)
    
imgs[0].shape

scale images and stack them

In [ ]:
scale = 100
fps = 50
original_width = imgs[0].shape[0]
original_height = imgs[0].shape[1]

dims = (int(original_width * scale / 100), int(original_height * scale / 100))
print("Scale every frame to {d[1]} * {d[0]}".format(d=dims))

stackIMG = create_data_matrix_from_images(imgs, scale)
imgs = 0
stackIMG.shape, dims

In [ ]:
plt.axis('off')
plt.imshow(np.reshape(stackIMG[:, 1442], dims), cmap='gray');

original matrix

In [ ]:
plt.figure(figsize=(120, 120))
plt.axis('off')
plt.imshow(stackIMG[:halfIndex, :], cmap='gray')

In [ ]:
plt.figure(figsize=(120, 120))
plt.axis('off')
plt.imshow(stackIMG[halfIndex:, :], cmap='gray')

# Background Removal of Video 1

SVD

In [ ]:
stackIMG = stackIMG.astype(float)
U, Sigma, Vt = svds(stackIMG, k=1)

U.shape, Sigma.shape, Vt.shape

In [ ]:
low_rank = U @ np.diag(Sigma) @ Vt
U = []
Sigma = []
Vt = []
#stackIMG = []
#plt.axis('off')
#plt.imshow(np.reshape(low_rank[:, 1442], dims), cmap='gray')

In [ ]:
low_rank.shape

low rank matrix

In [ ]:
plt.figure(figsize=(120, 120))
plt.axis('off')
halfIndex = int(low_rank.shape[0]/2)
plt.imshow(low_rank[:halfIndex, :], cmap='gray');

In [ ]:
plt.figure(figsize=(120, 120))
plt.axis('off')
plt.imshow(low_rank[halfIndex:, :], cmap='gray')

In [ ]:
car = stackIMG - low_rank
#low_rank = []
#stackIMG = []

In [ ]:
plt.axis('off')
plt.imshow(np.reshape(low_rank[:, 1300], dims), cmap='gray')

car matrix

In [ ]:
plt.figure(figsize=(120, 120))
plt.axis("off")
halfIndex = int(car.shape[0]/2)
plt.imshow(car[:halfIndex, :], cmap='gray');

In [ ]:
plt.figure(figsize=(120, 120))
plt.axis("off")
plt.imshow(car[halfIndex:, :], cmap='gray');

In [ ]:
plt.imshow(np.reshape(car[:, 0], dims), cmap='gray');

# Evaluation of Video1

get ground truth

In [ ]:
truth = []
for imgName in range(1,1500):
    temp = scipy.misc.imresize(rgb2gray(np.array(Image.open("./dataset/video1/truth/"+str(imgName)+".png"))), scale)
    truth.append(temp)
truth[0].shape

In [ ]:
car_eval = car.reshape(dims + (-1,))
car.shape, car_eval.shape

In [ ]:
x = car_eval[...,1442]

plt.axis("off")
plt.imshow(x, cmap='gray');

getCar2 is faster

In [ ]:
%timeit  newX = getCar(x)
%timeit  newX = getCar2(x)
np.array_equal(getCar(x), getCar2(x))

In [ ]:
plt.axis("off")
plt.imshow(getCar2(car_eval[...,1300]), cmap='gray')

turn all images into black and white

In [ ]:
newCarEval = []
for i in range(0,car_eval.shape[-1]):
    temp = car_eval[..., i]
    newCarEval.append(getCar2(temp))
#car_eval = []
#temp = []
#truth = []
#car = []
newCarEval = np.array(newCarEval)
plt.axis("off")
plt.imshow(newCarEval[750, ...], cmap='gray')
newCarEval.shape

get dice coefficient

In [ ]:
diceCoef(truth, newCarEval)

make black and white video

In [ ]:
makeVideoFromImgs('video1_black_white', newCarEval, firstAsTime=True)

evaluate different scale & different k of dice coefficient

In [ ]:
#for scale in [100]:
for scale in [10,25,50,75,100]:
    fps = 50
    original_width = imgs[0].shape[0]
    original_height = imgs[0].shape[1]
    dims = (int(original_width * scale / 100), int(original_height * scale / 100))
    stackIMG = create_data_matrix_from_images(imgs, scale)
    stackIMG = stackIMG.astype(float)
    truth = []
    for imgName in range(1,1500):
        temp = scipy.misc.imresize(rgb2gray(np.array(Image.open("./dataset/video1/truth/"+str(imgName)+".png"))), scale)
        truth.append(temp)
    print(dims)
    print("______________________________________________")
    for kk in range(2,8):
        U, Sigma, Vt = svds(stackIMG, k=kk)
        low_rank = U @ np.diag(Sigma) @ Vt
        U = []
        Sigma = []
        Vt = []
        car = stackIMG - low_rank
        low_rank = []
        car_eval = car.reshape(dims + (-1,))
        car = []
        newCarEval = []
        for i in range(0,car_eval.shape[-1]):
            temp = car_eval[..., i]
            newCarEval.append(getCar2(temp))
        car_eval = []
        temp = []
        newCarEval = np.array(newCarEval)
        print(diceCoef(truth, newCarEval))

# Make Video 1

In [ ]:
car_frames = car.reshape(dims + (-1,))
car_frames.shape

In [ ]:
videoName = 'video1'
makeVideoFromImgs(videoName, car_frames, firstAsTime=False)

In [ ]:
video1 = mpe.VideoFileClip("./video1.mp4")
video1.ipython_display(width=500)

# Video 2

http://bmc.iut-auvergne.com/?page_id=24

121, PNG files with ground truth

# Read Images

In [ ]:
imgs = []
for imgName in range(1,1500):
    imgs.append(np.array(Image.open("./dataset/video2/input/"+str(imgName)+".png")))

In [ ]:
scale = 100
fps = 50
original_width = imgs[0].shape[0]
original_height = imgs[0].shape[1]

dims = (int(original_width * scale / 100), int(original_height * scale / 100))
print("Scale every frame to {d[1]} * {d[0]}".format(d=dims))

stackIMG = create_data_matrix_from_images(imgs, scale)
imgs = 0
stackIMG.shape, dims

In [ ]:
#plt.axis('off')
plt.imshow(np.reshape(stackIMG[:, 1442], dims), cmap='gray');

In [ ]:
plt.figure(figsize=(60, 60))
plt.axis('off')
plt.imshow(stackIMG, cmap='gray')

# Background Removal of Video 2

In [ ]:
stackIMG = stackIMG.astype(float)
U, Sigma, Vt = svds(stackIMG, k=4)
U.shape, Sigma.shape, Vt.shape

In [ ]:
low_rank = U @ np.diag(Sigma) @ Vt

In [ ]:
plt.axis('off')
plt.imshow(np.reshape(low_rank[:, 1442], dims), cmap='gray');

low rank matrix

In [ ]:
plt.figure(figsize=(60, 60))
plt.imshow(low_rank, cmap='gray');

In [ ]:
car = stackIMG - low_rank

In [ ]:
low_rank = []

In [ ]:
plt.figure(figsize=(120, 120))
plt.axis('off')
halfIndex = int(car.shape[0]/2)
plt.imshow(car[halfIndex:, :], cmap='gray');

car matrix

In [ ]:
plt.figure(figsize=(60, 60))
plt.imshow(car, cmap='gray');

In [ ]:
plt.axis('off')
plt.imshow(np.reshape(car[:, 1400], dims), cmap='gray');

# Evaluation 2

get ground truth images

In [ ]:
truth = []
for imgName in range(1,1500):
    temp = scipy.misc.imresize(rgb2gray(np.array(Image.open("./dataset/video2/truth/"+str(imgName)+".png"))), scale)
    truth.append(temp)
truth[0].shape

In [ ]:
car_eval = car.reshape(dims + (-1,))
car.shape, car_eval.shape

turn all images into black and white

In [ ]:
newCarEval = []
for i in range(0,car_eval.shape[-1]):
    temp = car_eval[..., i]
    newCarEval.append(getCar2(temp))
#car_eval = []
temp = []
#truth = []
#car = []
newCarEval = np.array(newCarEval)
plt.axis("off")
plt.imshow(newCarEval[750, ...], cmap='gray')
newCarEval.shape

In [ ]:
diceCoef(truth, newCarEval)

make black and white video

In [ ]:
makeVideoFromImgs('video2_black_white', newCarEval, firstAsTime=True)

In [ ]:
#for scale in [100]:
for scale in [10,25,50,75,100]:
    fps = 50
    original_width = imgs[0].shape[0]
    original_height = imgs[0].shape[1]
    dims = (int(original_width * scale / 100), int(original_height * scale / 100))
    stackIMG = create_data_matrix_from_images(imgs, scale)
    stackIMG = stackIMG.astype(float)
    truth = []
    for imgName in range(1,1500):
        temp = scipy.misc.imresize(rgb2gray(np.array(Image.open("./dataset/video2/truth/"+str(imgName)+".png"))), scale)
        truth.append(temp)
    print(dims)
    print("______________________________________________")
    for kk in range(3,8):
        U, Sigma, Vt = svds(stackIMG, k=kk)
        low_rank = U @ np.diag(Sigma) @ Vt
        U = []
        Sigma = []
        Vt = []
        car = stackIMG - low_rank
        low_rank = []
        car_eval = car.reshape(dims + (-1,))
        car = []
        newCarEval = []
        for i in range(0,car_eval.shape[-1]):
            temp = car_eval[..., i]
            newCarEval.append(getCar2(temp))
        car_eval = []
        temp = []
        newCarEval = np.array(newCarEval)
        print(diceCoef(truth, newCarEval))

# Make Video 2

In [ ]:
car_frames = car.reshape(dims + (-1,))
car_frames.shape

In [ ]:
videoName = 'video2'
makeVideoFromImgs(videoName, car_frames, firstAsTime=False)

In [ ]:
video1 = mpe.VideoFileClip("./video1.mp4")
video1.ipython_display(width=500)

### Reference
- https://zulko.github.io/moviepy/getting_started/working_with_matplotlib.html